In [ ]:
%matplotlib inline
import random, pylab, math, numpy, os
import mpl_toolkits.mplot3d

In this homework set, you will study the local Metropolis, the heatbath and the Cluster Monte Carlo algorithms for the two-dimensional Ising model on a square lattice with periodic boundary conditions. At the end of the homework session, you will have validated all three programs against known analytic results. furthermore, you will have compared the results to a classic result of Ferdinand and Fisher, from 1969, analyzed the time-behavior of the cluster algorithm of Wolff (1989), and started to understand Propp and Wilson's coupling from the past approach.

# A

## A1

Here, you implement the local Metropolis algorithm, and validate it through the comparison with an
exactly known result. In a real-life application, this validation would run for many days on a computer, to establish agreement with the exact result to 5 or 6 significant digits.

The local Metropolis algorithm for the Ising model is given below.

In [ ]:
def energy(S, N, nbr):
    E = 0.0
    for k in range(N):
        E -= S[k] * sum(S[nn] for nn in nbr[k])
    return 0.5 * E

L = 6
N = L * L
nbr = {i : ((i // L) * L + (i + 1) % L, (i + L) % N,
            (i // L) * L + (i - 1) % L, (i - L) % N) 
       for i in range(N)}

T = 2.0
filename = 'local_'+ str(L) + '_' + str(T) + '.txt'
S = [random.choice([1, -1]) for k in range(N)]
nsteps = N * 100
beta = 1.0 / T
Energy = energy(S, N, nbr)
E = []

for step in range(nsteps):
    k = random.randint(0, N - 1)
    delta_E = 2.0 * S[k] * sum(S[nn] for nn in nbr[k])
    if random.uniform(0.0, 1.0) < math.exp(-beta * delta_E):
        S[k] *= -1
        Energy += delta_E
    E.append(Energy)
E_mean = sum(E)/ len(E)

print sum(E) / len(E) / N, 'mean energy'

Greatly increase the number of iterations of the algorithm.

Run it for L=6 and for T = 2.0.

The exact value is known from exact enumeration: E/N =-1.747

# QUESTION

1.  Communicate the results obtained in four independent runs of the algorithms (no calculation of error bars required).

## A2

Modify your program so that it allows you to read in and write out configurations and also to print configurations on the screen. For simplicity, this program is provided, you are free run and modify it.

In [ ]:
def x_y(k, L):
    y = k // L
    x = k - y * L
    return x, y

L = 128
N = L * L
nbr = {i : ((i // L) * L + (i + 1) % L, (i + L) % N,
            (i // L) * L + (i - 1) % L, (i - L) % N)
       for i in range(N)}

T = 1.0
filename = 'local_'+ str(L) + '_' + str(T) + '.txt'
if os.path.isfile(filename):
    f = open(filename, 'r')
    S = []
    for line in f:
        S.append(int(line))
    f.close()
    print 'starting from file', filename
else:
    S = [random.choice([1, -1]) for k in range(N)]
    print 'starting from scratch'
    
nsteps = N * 100
beta = 1.0 / T

for step in range(nsteps):
    k = random.randint(0, N - 1)
    delta_E = 2.0 * S[k] * sum(S[nn] for nn in nbr[k])
    if random.uniform(0.0, 1.0) < math.exp(-beta * delta_E):
        S[k] *= -1
        

conf = [[0 for x in range(L)] for y in range(L)]
for k in range(N):
    x, y = x_y(k, L)
    conf[x][y] = S[k]

In [ ]:
pylab.imshow(conf, extent = [0, L, 0, L], interpolation='nearest')
pylab.set_cmap('hot')
pylab.colorbar()
pylab.title('Local_'+ str(T) + '_' + str(L))
pylab.savefig('Local_'+ str(T) + '_' + str(L)+ '.png')
pylab.show()
f = open(filename, 'w')
for a in S:
    f.write(str(a) + '\n')
f.close()

From a random initial configuration, run this program at high temperature $T = 3.0$ (L=128) and at the critical temperature $T_{crit} = 2.27$ (L=128). Run it for sufficiently long, so that you have reached what you would consider as the $t \to \infty$ limit. (At $T_{crit}$, you should run your code for at least a few minutes). Notice that subsequent runs of this program realize a Markov chain (i.e., three runs with 100 + 100 + 100 iterations = 1 run with 300 iterations).

# QUESTIONS

1. Display typical configurations that you have obtained and comment on what you see. 
2. At low temperature, T = 1.0, run your code for L=32, first for 10 N iterations, then again, several times for 10 N iterations, then several times for 100 N iterations (if you see a stripe, run your program longer, up to several ties 1000 N or several times 10000 N iterations). You should observe that, after a quite long time, your simulation goes from a state with domain walls into a purely ferromagnetic state (essentially all spins +1 or essentially all spins -1). Display two or three pictures at different times. Do you observe (running for several times 10000 N iterations) that the local Metropolis algorithm flips between configurations of negative overall magnetization (mostly black) and configurations of positive overall magnetization (mostly white)?
3. At low temperature, T = 1.0 (as before), run your code for L=128 and display a configuration. Explain what you observe. In particular, do you observe that the system becomes homogeneous (mostly white or mostly black) on the available time scales?

# B

In this section, you run the Wolff cluster algorithm, one of the great (but little understood) algorithms of statistical physics.

## B1

The Wolff cluster Monte Carlo algorithm for the Ising model is implemented below.

Familiarize yourself with how it works before going on. Explain in a few words what the Pocket stands for.

In [ ]:
def energy(S, N, nbr):
    E = 0.0
    for k in range(N):
        E -= S[k] * sum(S[nn] for nn in nbr[k])
    return 0.5 * E

L = 6
N = L * L
nbr = {i : ((i // L) * L + (i + 1) % L, (i + L) % N,
            (i // L) * L + (i - 1) % L, (i - L) % N)
       for i in range(N)}

T = 2.0
p = 1.0 - math.exp(-2.0 / T)
nsteps = 10000
S = [random.choice([1, -1]) for k in range(N)]
E = [energy(S, N, nbr)]

for step in range(nsteps):
    k = random.randint(0, N - 1)
    Pocket, Cluster = [k], [k]
    while Pocket != []:
        j = random.choice(Pocket)
        for l in nbr[j]:
            if S[l] == S[j] and l not in Cluster and random.uniform(0.0, 1.0) < p:
                Pocket.append(l)
                Cluster.append(l)
        Pocket.remove(j)
    for j in Cluster:
        S[j] *= -1
    E.append(energy(S, N, nbr))
print sum(E)/ len(E) / N

Greatly increase the number of iterations of the algorithm. 

Run it for L=6 and T = 2.0.

Check that you recover the exact value for the mean energy $E/N(T=2, L=6) = -1.747$ (known from exact enumeration).

# QUESTIONS

1. Communicate the results obtained in four independent runs of the algorithms.
2. Is it necessary to pick a random element j of the pocket (element that we eliminate from the pocket through Pocket.remove(j)), or can we simply pop the last element of Pocket (j = Pocket.pop())?
3. Modify then run the program modified according to the suggestion of the preceding question (note that it has one less line, the Pocket.remove() line is no longer needed), and explain your observations.

## B2 

The Ising model was solved exactly by Onsager, in 1944, for the infinite lattice. The exact solution for the finite lattice was obtained by Kaufman in 1949 . Based on her paper, Ferdinand and Fisher, in 1969, computed the specific heat for finite lattices with periodic boundary conditions, exactly the system that we consider, and the result is shown in Fig1 of their paper, which you will find in your repo, in the "Figures" folder below the current one.

Incorporate into the cluster algorithm the read-in, write-out part of section A2, which allows you to do a 'warm start' (that is, discard some initial data). Run this modified cluster algorithm for lattices of size L=2, 4, 8, 16, 32 and check Ferdinand and Fisher's analytical results. To do so, implement a few lines analogous to:

In [ ]:
E_mean = sum(E)/ len(E)
E2_mean = sum(a ** 2 for a in E) / len(E)
cv = (E2_mean - E_mean ** 2 ) / N / T ** 2

(Remember the definition of the specific heat given in class).

# QUESTION

1. Communicate your results (without error bars, just quote the results of one or two runs, after a warm start, for each value of the parameters) at the critical temperature T = 2.27.
NB: we just need rough agreement.
2. Can you confirm that observables such as the specific heat for the Ising model in the transition region exhibit a strong finite size effect?

# C

In this section, you run the heatbath algorithm.

## C1

The heatbath algorithm for the Ising model is implemented in the cell below. Familiarize yourself with how it works.

In [ ]:
L = 6
N = L * L
nbr = {i : ((i // L) * L + (i + 1) % L, (i + L) % N,
            (i // L) * L + (i - 1) % L, (i - L) % N) 
       for i in range(N)}

nsteps = 10 * N
T = 2
beta = 1.0 / T

S = [random.choice([-1, 1]) for site in range(N)]
E = -0.5 * sum(S[k] * sum(S[nn] for nn in nbr[k]) for k in range(N))
Energies = []

for step in range(nsteps):
    k = random.randint(0, N - 1)
    Upsilon = random.uniform(0.0, 1.0)
    h = sum(S[nn] for nn in nbr[k])
    Sk_old = S[k]
    S[k] = -1
    if Upsilon < 1.0 / (1.0 + math.exp(-2.0 * beta * h)):
        S[k] = 1
    if S[k] != Sk_old:
        E -= 2.0 * h * S[k]
    Energies.append(E)
print sum(Energies) / len(Energies) / N

Greatly increase the number of iterations of the algorithm.

Run it for L=6 and for T = 2.0. Check that you recover the exact value for the mean energy E/N(T=2, L=6) = -1.747.

# QUESTION

1. Communicate the results obtained in four independent runs of the algorithm.

## C2 

Modify the heatbath algorithm so that it does **two** computations in parallel, as discussed during the lecture:
one starting from the all plus spin configuration, and one from the all minus configuration. For simplicity,
the implementation is provided. You are free run and modify it, but familiarize yourself thoroughly with this program.


In [ ]:
L = 32
N = L * L
nbr = {i : ((i // L) * L + (i + 1) % L, (i + L) % N,
            (i // L) * L + (i - 1) % L, (i - L) % N)
       for i in range(N)}

T = 5.0
beta = 1.0 / T
t_coup = []
for iter in range(10):
    print iter
    S0 = [1] * N
    S1 = [-1] * N
    step = 0
    while True:
        step += 1
        k = random.randint(0, N - 1)
        Upsilon = random.uniform(0.0, 1.0)
        h = sum(S0[nn] for nn in nbr[k])
        S0[k] = -1
        if Upsilon < 1.0 / (1.0 + math.exp(-2.0 * beta * h)):
            S0[k] = 1
        h = sum(S1[nn] for nn in nbr[k])
        S1[k] = -1
        if Upsilon < 1.0 / (1.0 + math.exp(-2.0 * beta * h)):
            S1[k] = 1
        if step % N == 0:
            n_diff = sum(abs(S0[i] - S1[i]) for i in range(N))
            if n_diff == 0:
                t_coup.append(step / N)
                break
print t_coup
print sum(t_coup) / len(t_coup)

Compute the time (in number of "sweeps", i.e. in number of steps / N) at which the algorithm couples, that is, at which the difference between the configurations generated in one run and the other falls to zero. Plot this coupling time (averaged over 10 runs with different random numbers) for L = 32 at temperature T= 5.0, 4.0, 3.0, 2.5, 2.4, 2.3. **Attention**: run T=2.3 only if you have a lot of CPU time available...In the Ising model, the coupling time is, up to a logarithmic factor in N, equal to the correlation time, so this coupling time informs you of the order of magnitude of the correlation time. 

# QUESTIONS

1. Describe what you see.
2. Ferdinand and Fisher, in a footnote of their 1969 paper discussed earlier, compared their exact calculations with the earliest Monte Carlo calculations on the Ising model. Take a look at the footnote 20 in the paper, p841. In the light of your exact bounds on the correlation time, can you explain the insufficient agreement between theory and numerics reported by Ferdinand and Fisher?

The algorithm in C2 couples, and its coupling time provides a rigorous limit on the convergence time. Nevertheless, the configuration obtained is not really a random configuration, simply because the coupling time itself has fluctuations. For your information and enjoyment, we provide here a rudimentary yet exact algorithm following the "coupling from the past principle" that, rather than simulating from time t=0 to a coupling time t_coup, mimicks a simulation from $t= -\infty$ to $t=0$.

In [ ]:
L = 16
N = L * L
nbr = {i : ((i // L) * L + (i + 1) % L, (i + L) % N,
            (i // L) * L + (i - 1) % L, (i - L) % N) 
       for i in range(N)}

nsteps = 10
T = 3.0
beta = 1.0 / T
S0 = [1] * N
S1 = [-1] * N
k = {}
Upsilon = {}
n_diff = 10

while n_diff != 0:
    nsteps *= 2
    for step in range(-nsteps, 0):
        if step not in k:
            k[step] = random.randint(0, N - 1)
            Upsilon[step] = random.uniform(0.0, 1.0)
        h = sum(S0[nn] for nn in nbr[k[step]])
        S0[k[step]] = -1
        if Upsilon[step] < 1.0 / (1.0 + math.exp(-2.0 * beta * h)):
            S0[k[step]] = 1
        h = sum(S1[nn] for nn in nbr[k[step]])
        S1[k[step]] = -1
        if Upsilon[step] < 1.0 / (1.0 + math.exp(-2.0 * beta * h)):
            S1[k[step]] = 1
    n_diff = 0
    for i in range(N):
        if S0[i] != S1[i]: 
            n_diff += 1
    print nsteps, n_diff

The configuration at t=0 is a perfect sample, following the coupling from the past idea of Propp and
Wilson (1996). You can see that an infinite simulation, from t = -infty up to t = 0, would output the sample
at t=0.


# QUESTION

Np points, no question asked - purely for scientific interest.